In [1]:
import polars as pl
import numpy as np

In [7]:
presidents = (
    pl.read_csv("../data/1976-2020-president.csv", null_values=["NA"])
    .filter(pl.col("year") >= 2016)
    .group_by(["year", "state"])
    .agg(pl.col("candidate").sort_by("totalvotes").first())
)

In [8]:
presidents

year,state,candidate
i64,str,str
2020,"""NEBRASKA""","""TRUMP, DONALD J."""
2016,"""NEW MEXICO""","""CLINTON, HILLARY"""
2020,"""MARYLAND""","""TRUMP, DONALD J."""
2020,"""NORTH DAKOTA""","""TRUMP, DONALD J."""
2016,"""IDAHO""","""TRUMP, DONALD J."""
…,…,…
2016,"""IOWA""","""TRUMP, DONALD J."""
2016,"""RHODE ISLAND""","""CLINTON, HILLARY"""
2020,"""MISSISSIPPI""","""BIDEN, JOSEPH R. JR"""


In [24]:
grouped = presidents.group_by(["year", "candidate"]).all()

coalitions = grouped["state"]
coalitions

state
list[str]
"[""NEW MEXICO""]"
"[""IDAHO"", ""MISSISSIPPI"", … ""IOWA""]"
"[""DELAWARE"", ""GEORGIA""]"
"[""MONTANA"", ""NEVADA"", ""NEW HAMPSHIRE""]"
"[""WEST VIRGINIA"", ""ALABAMA"", … ""MISSISSIPPI""]"
"[""NEW MEXICO"", ""DISTRICT OF COLUMBIA"", … ""HAWAII""]"
"[""SOUTH CAROLINA"", ""ARKANSAS""]"
"[""NEBRASKA"", ""MARYLAND"", … ""MISSOURI""]"
"[""UTAH""]"


In [28]:
df = presidents.sort("year")

# Create a voting pattern for each state by collecting the candidates they voted for, ordered by year
state_votes = df.group_by("state").all().rename({"candidate": "voting_pattern"})

# Group states by their voting patterns to find coalitions
coalitions = state_votes.group_by("voting_pattern").all().rename({"state": "states"})

# Extract the list of coalitions
coalition_list = coalitions["states"].to_list()

# PANDAS BELOW!

In [25]:
import pandas as pd

In [43]:
df = pd.read_csv("../data/1976-2020-president.csv")
df = df[df["year"] >= 2016]

In [44]:
# Select the columns relevant for comparison
comparison_cols = ["year", "office", "state", "candidate"]

# Group votes by state, year, and office, and aggregate candidates into tuples
state_patterns = (
    df.groupby(["state", "year", "office"])["candidate"]
    .apply(
        lambda x: tuple(sorted(str(c) for c in x if pd.notna(c)))
    )  # Convert to string and sort
    .reset_index()
)

# Pivot the table to align states' voting patterns across years and offices
pivot_table = state_patterns.pivot_table(
    index="state",
    columns=["year", "office"],
    values="candidate",
    aggfunc=lambda x: x,  # Pass the already-processed tuple
)

# Find identical voting patterns across states
block_patterns = pivot_table.apply(lambda row: tuple(row), axis=1).reset_index()
block_patterns.columns = ["state", "voting_pattern"]

# Group by voting pattern to find states with identical patterns
state_blocks = (
    block_patterns.groupby("voting_pattern")["state"].apply(list).reset_index()
)
state_blocks.columns = ["voting_pattern", "states_in_block"]

# Filter for blocks with more than one state
blocks_with_multiple_states = state_blocks[
    state_blocks["states_in_block"].apply(len) > 1
]

# Print the result
print(blocks_with_multiple_states)

                                       voting_pattern  \
44  ((CLINTON, HILLARY, JOHNSON, GARY, STEIN, JILL...   

                states_in_block  
44  [ALABAMA, KANSAS, NEBRASKA]  
